## TO DO:
- [-] Take rough number of images to be downloaded as input. (Rough because few images will be corrupt and deleted later on.) - MAY NOT NEED THIS, SEE BELOW.
- [x] The make_image_dataset function should be able to take Lists as input. When taking list inputs, need to create folders within destination folder to segregate the images.

## Getting more data:
1. Tagged Anime Illustrations: <br>
https://www.kaggle.com/mylesoneill/tagged-anime-illustrations/home - Will need to clone this directly into GCP. 36 GB is too much for my pleb laptop.

2. Anime Faces : <br>
http://www.nurs.or.jp/~nagadomi/animeface-character-dataset/

3. Check out the links above they lead to tabular anime data as well.

### We can this the dataset gathered for Anime Identification. That is, if the above falls short.

## Lesson learnt:
## Save all FAST AI notebooks that you edit to your local storage.

In [14]:
#!pip install pyvirtualdisplay
#!pip install xvfbwrapper
#!sudo apt-get install xvfb

In [20]:
from imutils import paths
import argparse
import requests
import cv2
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
import glob
import time
from pyvirtualdisplay import Display
from threading import *

### Below list has characters due to which we will only get 97 categories when classifying anime.

In [21]:
#Anime_List = ["Cowboy Bebop", "Steins Gate", "Maison Ikkoku", "Baccano", "Mushishi", "The Rose of Versailles", "Berserk", "Now and Then Here and There", "Legend of the Galactic Heroes", "Usagi Drop", "Rurouni Kenshin Wandering Samurai", "Guardian of the Sacred Spirit", "Planetes", "Great Teacher Onizuka", "The Irresponsible Captain Tylor", "Lupin the 3rd", "Death Note", "Monster", "Elfen Lied", "Hunter x Hunter", "Neon Genesis Evangelion", "City Hunter", "Ghost in the Shell Stand Alone Complex", "Code Geass Lelouch of the Rebellion", "Tomorrow's Joe", "Black Lagoon", "Darker Than Black Gemini of the Meteor", "Yu Yu Hakusho Ghost Files", "Ranma ½", "Samurai Champloo", "Gungrave", "Tengen toppa gurren lagann", "Claymore", "Fullmetal Alchemist", "Hellsing Ultimate", "Kimagure Orange Road", "Future Diary", "Eureka Seven", "Katanagatari", "Princess Tutu", "Tenchi Muyô!", "Spice and Wolf", "Super Dimensional Fortress Macross", "Bakemonogatari", "School Rumble", "Hajime no ippo", "Revolutionary Girl Utena", "Natsume's Book of Friends", "Inuyasha", "The Twelve Kingdoms", "Kaiba", "Escaflowne", "Koi kaze", "When They Cry", "Ergo Proxy", "Durarara!!", "Mobile Suit Gundam", "FLCL", "Future Boy Conan", "The Count of Monte Cristo", "Welcome to the NHK!", "Nobody's Boy Remi", "The Slayers", "Gambling Apocalypse Kaiji", "Kimi ni todoke", "Serial Experiments Lain", "Ouran High School Host Club", "Les Miserables Shoujo Cosette", "Michiko and Hatchin", "Romeo's Blue Skies", "Mononoke", "Phantom Requiem for the Phantom", "Rainbow Nishakubou no shichinin", "The Mysterious Cities of Gold", "Goodbye, Mr", "Anne of Green Gables", "Space Pirate Captain Harlock", "Touch", "Chobits", "Treasure Island", "Urusei yatsura", "Wolf's Rain", "Fist of the North Star", "Seitokai yakuindomo", "Ef A Tale of Memories", "Trigun", "Haibane renmei", "Toradora!", "Kaleido Star", "Blood+", "Golden Boy", "JoJo's Bizarre Adventure", "The Melancholy of Haruhi Suzumiya", "XXXHOLiC", "Nodame kantâbire", "Crest of the Stars", "Clannad", "Texhnolyze", "Yojohan shinwa taikei"]

### This is the new, updated list, we will need to replace spaces with underscores once we download the images. Made a few other fixes so that we find only the desired anime.

In [22]:
#Anime_List_Fixed = ["Cowboy Bebop","Steins Gate","Maison Ikkoku","Baccano","Mushishi","The Rose of Versailles","Berserk","Now and Then Here and There","Legend of the Galactic Heroes","Usagi Drop","Rurouni Kenshin Wandering Samurai","Guardian of the Sacred Spirit","Planetes","Great Teacher Onizuka","The Irresponsible Captain Tylor","Lupin the 3rd","Death Note","Monster","Elfen Lied","Hunter x Hunter","Neon Genesis Evangelion","City Hunter","Ghost in the Shell Stand Alone Complex","Code Geass Lelouch of the Rebellion","Tomorrows Joe","Black Lagoon","Darker Than Black: Gemini of the Meteor","Yu Yu Hakusho Ghost Files","Ranma","Samurai Champloo","Gungrave","Tengen toppa gurren lagann","Claymore","Fullmetal Alchemist","Hellsing Ultimate","Kimagure Orange Road","Future Diary","Eureka Seven","Katanagatari","Princess Tutu","Tenchi Muyo","Spice and Wolf","Super Dimensional Fortress Macross","Bakemonogatari","School Rumble","Hajime no ippo","Revolutionary Girl Utena","Natsumes Book of Friends","Inuyasha","The Twelve Kingdoms","Kaiba","Escaflowne","Koi kaze","When They Cry","Ergo Proxy","Durarara","Mobile Suit Gundam","FLCL","Future Boy Conan","The Count of Monte Cristo","Welcome to the NHK","Nobodys Boy Remi","The Slayers","Gambling Apocalypse Kaiji","Kimi ni todoke","Serial Experiments Lain","Ouran High School Host Club","Les Miserables Shoujo Cosette","Michiko and Hatchin","Romeos Blue Skies","Mononoke","Phantom Requiem for the Phantom","Rainbow Nishakubou no shichinin","The Mysterious Cities of Gold","Sayonara Zetsubou Sensei","Anne of Green Gables anime","Space Pirate Captain Harlock","Touch","Chobits","Treasure Island","Urusei yatsura","Wolfs Rain","Fist of the North Star","Seitokai yakuindomo","Ef A Tale of Memories","Trigun","Haibane renmei","Toradora","Kaleido Star","Blood plus","Golden Boy","JoJos Bizarre Adventure","The Melancholy of Haruhi Suzumiya","XXXHOLiC","Nodame Cantabile","Crest of the Stars","Clannad","Texhnolyze","Yojohan shinwa taikei"]

In [23]:
images_to_download_list = ["Great Teacher Onizuka", "Akage no Anne", "Sayonara Zetsubou Sensei"]

In [24]:
#list_of_files = glob.glob(r"C:\Users\kmoudgalya\Downloads\*") # * means all if need specific format then *.csv

In [25]:
def make_image_dataset_v3(images_to_download, dest_folder = r"./data_v3_old/Anime_Recogntn_Images/", downloads_folder= r"/home/jupyter/Downloads/"):    
    #images_to_download = input("Which images do you want to download? ")
    #dest_folder = input("Where do you want the downloaded images? ")
    #downloads_folder = input("Enter the path to your downloads folder. ")
    for image_name in images_to_download:
        print("Downloading Images of {}".format(image_name))
        
        os.mkdir(os.path.sep.join([dest_folder, image_name]))
        dest_folder_new = os.path.sep.join([dest_folder, image_name])

        options = webdriver.ChromeOptions()
        #options.add_argument('headless')
        #options.add_argument('window-size=1200x600')   
        display = Display(visible=0, size=(1200, 600))
        display.start()
        
        driver = webdriver.Chrome(executable_path = "/home/jupyter/chromedriver.exe", options=options)
        
        driver.get("https://www.google.com/search?tbm=isch&q=" + image_name)

        urls = driver.execute_script(r'''urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
        window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));''')
        time.sleep(4)
        
        driver.quit()
        display.stop()

        list_of_files = glob.glob(os.path.sep.join([downloads_folder, "/*"]))
        latest_file = max(list_of_files, key=lambda x: os.stat(x).st_mtime)

        rows = open(latest_file).read().strip().split("\n")
        total = 0


        # loop the URLs
        for url in rows:
            try:
                # try to download the image
                r = requests.get(url, timeout=10)

                # save the image to disk
                p = os.path.sep.join([dest_folder_new, "{}.jpg".format(str(total).zfill(8))])
                f = open(p, "wb")
                f.write(r.content)
                f.close()

                # update the counter
                print("[INFO] downloaded: {}".format(p))
                total += 1
                
            # handle if any exceptions are thrown during the download process
            except:
                print("[INFO] error downloading {}...skipping".format(p))


        # loop over the image paths we just downloaded
        for imagePath in paths.list_images(dest_folder_new):
            # initialize if the image should be deleted or not
            delete = False

            # try to load the image
            try:
                image = cv2.imread(imagePath)

                # if the image is `None` then we could not properly load it
                # from disk, so delete it
                if image is None:
                    delete = True

            # if OpenCV cannot load the image then the image is likely
            # corrupt so we should delete it
            except:
                print("Except")
                delete = True

            # check to see if the image should be deleted
            if delete:
                print("[INFO] deleting {}".format(imagePath))
                os.remove(imagePath)

    print("Images downloaded. Let the Machine Learning begin.")
    

In [26]:
start_time = time.time()
make_image_dataset_v3(images_to_download_list)
print('Entire job took:',time.time() - start_time)

[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000000.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000001.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000002.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000003.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000004.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000005.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000006.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000007.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000008.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000009.jpg
[INFO] downloaded: ./data_v3_old/Anime_Recogntn_Images//Great Teacher Onizuka/00000010.jpg

In [43]:
# filelist = os.listdir(os.getcwd())
# filelist = glob.glob(os.path.sep.join([r"/home/jupyter/Downloads/", "*"]))
# newest = max(filelist, key=lambda x: os.stat(x).st_mtime)
# print(newest)

/home/jupyter/Downloads/download (1)


In [4]:
for i in images_to_download_list:
    print(i)

naruto
cowboy bebop


In [18]:
!rm -r ./trial
!mkdir ./trial

[INFO] downloaded: ./trial/n/00000099.jpg
[INFO] deleting ./trial/n/00000035.jpg
[INFO] deleting ./trial/n/00000078.jpg
[INFO] deleting ./trial/n/00000092.jpg
[INFO] deleting ./trial/n/00000064.jpg


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-cb028374cf37>", line 79, in make_image_dataset_v3
    os.remove(imagePath)
FileNotFoundError: [Errno 2] No such file or directory: './trial/n/00000064.jpg'



[INFO] error downloading ./trial/c/00000042.jpg...skipping
[INFO] error downloading ./trial/c/00000042.jpg...skipping
[INFO] error downloading ./trial/c/00000042.jpg...skipping
[INFO] error downloading ./trial/c/00000042.jpg...skipping
[INFO] error downloading ./trial/c/00000042.jpg...skipping
[INFO] error downloading ./trial/c/00000042.jpg...skipping
[INFO] error downloading ./trial/c/00000042.jpg...skipping
[INFO] error downloading ./trial/c/00000042.jpg...skipping


### Multi-threaded download:

In [17]:
start_time = time.time()
threadlist = []
for j in images_to_download_list:
    print(j)
    t = Thread(target=make_image_dataset_v3, args=j)
    
    t.start()
    threadlist.append(t)
    
for b in threadlist:
   b.join()
    
print('Entire job took:',time.time() - start_time)

naruto
cowboy bebop
[INFO] downloaded: ./trial/c/00000000.jpg
[INFO] downloaded: ./trial/c/00000001.jpg
[INFO] downloaded: ./trial/c/00000002.jpg
[INFO] downloaded: ./trial/c/00000003.jpg
[INFO] downloaded: ./trial/c/00000004.jpg
[INFO] downloaded: ./trial/c/00000005.jpg
[INFO] downloaded: ./trial/n/00000000.jpg
[INFO] downloaded: ./trial/n/00000001.jpg
[INFO] downloaded: ./trial/c/00000006.jpg
[INFO] downloaded: ./trial/n/00000002.jpg
[INFO] downloaded: ./trial/n/00000003.jpg
[INFO] downloaded: ./trial/n/00000004.jpg
[INFO] downloaded: ./trial/n/00000005.jpg
[INFO] downloaded: ./trial/c/00000007.jpg
[INFO] downloaded: ./trial/n/00000006.jpg
[INFO] downloaded: ./trial/c/00000008.jpg
[INFO] downloaded: ./trial/n/00000007.jpg


KeyboardInterrupt: 

[INFO] downloaded: ./trial/n/00000008.jpg
[INFO] downloaded: ./trial/n/00000009.jpg
[INFO] downloaded: ./trial/n/00000010.jpg
[INFO] downloaded: ./trial/n/00000011.jpg
[INFO] downloaded: ./trial/n/00000012.jpg
[INFO] downloaded: ./trial/n/00000013.jpg
[INFO] downloaded: ./trial/c/00000009.jpg
[INFO] downloaded: ./trial/c/00000010.jpg
[INFO] downloaded: ./trial/c/00000011.jpg
[INFO] downloaded: ./trial/c/00000012.jpg
[INFO] downloaded: ./trial/n/00000014.jpg
[INFO] downloaded: ./trial/n/00000015.jpg
[INFO] downloaded: ./trial/c/00000013.jpg
[INFO] downloaded: ./trial/n/00000016.jpg
[INFO] downloaded: ./trial/n/00000017.jpg
[INFO] downloaded: ./trial/n/00000018.jpg
[INFO] downloaded: ./trial/n/00000019.jpg
[INFO] downloaded: ./trial/n/00000020.jpg
[INFO] downloaded: ./trial/n/00000021.jpg
[INFO] downloaded: ./trial/n/00000022.jpg
[INFO] downloaded: ./trial/n/00000023.jpg
[INFO] downloaded: ./trial/c/00000014.jpg
[INFO] downloaded: ./trial/c/00000015.jpg
[INFO] downloaded: ./trial/n/00000

### The script simply pauses if one puts down the laptop lid and resumes when the user logs back in to the laptop. Phew

### 100x100 images (10000) take around 6-7 hours to be downloaded.